**Grupo:**

*  Teo Argañaraz (materia completa)
*  Marilina Trevisan (materia completa)
*  Ana Legaspi (materia completa)
*  Florencia Molina (materia completa)
*  Elena Rufeil Fiori (materia completa)


<h1><center><font size="15">
Análisis de Grafos
</font></center></h1>

<center><b>Nahuel Almeira</b></center>

# Práctico

El trabajo práctico de la materia consiste en el análisis de un conjunto de datos extraído de Twitter. La idea es emplear los conceptos de grafos vistos en clase sobre un caso real de actualidad.

## Dataset

El dataset consiste en un conjunto de hilos de tweets, con un total de ~150000 tweets, extraídos entre Enero y Marzo de 2021. La temática de los mismos está referida a la vacunación contra el covid-19 en Argentina.

Pueden descargar el dataset del siguiente [link](https://drive.google.com/file/d/1X_qKsE8muAnom2tDX4sLlmBAO0Ikfe_G/view?usp=sharing).

### Campos

- **created_at:** Fecha del tweet
- **id_str:** ID del tweet
- **full_text:** Contenido del tweet
- **in_reply_to_status_id:** ID del tweet inmediatamente anterior en el hilo
- **in_reply_to_user_id:** Autor del tweet inmediatamente anterior en el hilo
- **user.id:** Autor del tweet
- **user_retweeters:** Lista de ID de usuarios que retweetearon el tweet
- **sentiment:** Etiquetado manual que indica el sentimiento o intención del tweet con respecto al tweet anterior en el hilo

## Configuración inicial

In [1]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.set_context('talk')

## Descargar el csv con los datos en este directorio
#DATA_DIR = Path('../data/twitter')
#INPUT_FILE = DATA_DIR / 'vacunas.csv'

## Creamos el directorio en caso de que no exista
#DATA_DIR.mkdir(parents=True, exist_ok=True)

## Montamos el archivo desde el drive
#from google.colab import drive
#drive.mount('/content/drive')
#SHARED_FOLDER_ALIAS = 'Diplo_grafos'
#DATASETS_PATH = '/content/drive/MyDrive/' + SHARED_FOLDER_ALIAS + '/data_preprocessed/'

## Importamos la base desde la computadora
from google.colab import files
uploaded = files.upload()

Saving vacunas.csv to vacunas.csv


### Cargamos el dataset

In [2]:
dtypes = {
    'id_str': str,
    'full_text': str,
    'in_reply_to_status_id': str,
    'in_reply_to_user_id': str,
    'user.id': str
}
df = pd.read_csv('vacunas.csv', dtype=dtypes).dropna(subset=['user_retweeters'])
df['user_retweeters'] = df['user_retweeters'].apply(lambda x: [str(elem) for elem in eval(x)])
print(df.shape)
df.head()

(155123, 8)


,created_at,id_str,full_text,in_reply_to_status_id,in_reply_to_user_id,user.id,user_retweeters,sentiment
0,Sat Feb 20 03:09:10 +0000 2021,1362962469749153792,Seguimos esperando el comunicado de @norabar r...,NaN,NaN,2737379453,"[2258074658, 159909978, 105301854, 290671142, ...",NaN
1,Sat Feb 20 03:19:59 +0000 2021,1362965193509265417,@Clon_43 @norabar Nora estaba indignada porque...,1362962469749153792,2737379453,32718111,[],NaN
2,Mon Feb 22 23:55:08 +0000 2021,1364000806740111363,"Bueno, Alberto dijo Salud o Economía. La salud...",NaN,NaN,252168075,"[1238117630696972289, 37232479, 12792246571247...",NaN
3,Tue Feb 23 00:09:14 +0000 2021,1364004354374696963,@spitta1969 Tuit del mes Spitta,1364000806740111363,252168075,1156346340802224128,[],NaN
4,Tue Feb 23 00:00:17 +0000 2021,1364002100364128260,@spitta1969 Estas onfire,1364000806740111363,252168075,153663816,[],NaN


### Observamos algunos ejemplos

In [3]:
idx = 0
print('Texto:', df.full_text.values[idx])
print('Retweets:', len(df.user_retweeters.values[idx]))

Texto: Seguimos esperando el comunicado de @norabar repudiando la situación respecto del gobierno y el tema vacunas. Seamos pacientes que con esto de la pandemia anda con mucho "laburo".
Retweets: 9


In [4]:
idx = 376
print('Text:', df.full_text.values[idx])
print('Retweets:', len(df.user_retweeters.values[idx]))

Text: Todo lo que hay que entender sobre la decisión –o no– de poner más vacunas en más brazos (por ejemplo, usar las 1º dosis en muchos y si es necesario retrasar la 2º) está en esta excelente nota de Nora Bär. https://t.co/A0I03DyxgO
Retweets: 48


### Calculamos la cantidad de hilos

In [5]:
roots = df[df['in_reply_to_user_id'].isna()]
roots.shape

(3174, 8)

## Actividades

### Primera parte

#### **1. Construcción del grafo** 

Construir el **grafo de retweets**, definido de la siguiente manera:

- Tipo de grafo: Dirigido
- Nodos: ID de los usuarios
- Enlaces: (Usuario A) ---> (Usuario B) si B retweeteó algún tweet de A

Con estos datos, el grafo debería tener alrededor de 40000 nodos y 90000 enlaces.

Considerar la versión no dirigida del grafo y estudiar su conectividad. Si existe una única "componente gigante", realizar el resto de las actividades sobre ella, en lugar de sobre el grafo completo.

Calcular las siguientes métricas globales del grafo:

- Grado medio
- Asortatividad
- Transitividad
- Coeficiente de clustering de Watts-Strogatz

**Opcional:** Comparar las métricas calculadas anteriormente con las de un grafo aleatorio con la misma distribución de grado. Pueden utilizar para ello este [método](https://networkx.org/documentation/stable/reference/generated/networkx.generators.degree_seq.configuration_model.html?highlight=configuration#networkx.generators.degree_seq.configuration_model). Con esto en mente, comentar si los valores obtenidos anteriormente difieren significativamente del caso aleatorio.


#### **2. Centralidad**

Calcular 5 métricas de centralidad de nodos. Graficar la distribución de cada una de ellas ¿Existe alguna correlación entre las distintas centralidades? 

Hacer un ranking con los 10 nodos más centrales para cada métrica. ¿Hay coincidencia entre los rankings?. ¿Qué características tienen los usuarios más centrales y sus respectivos tweets?

**Opcional:** Determinar si existe alguna correlación entre la centralidad de un nodo y su actividad en red social. Es decir, evaluar si los usuarios que más escriben son los más centrales o no.

#### **3. Comunidades**

Utilizar el algoritmo de Louvain con el parámetro "resolución" igual a 1. Caracterizar las comunidades halladas (cantidad, distribución de tamaños). Utilizar la modularidad y otras dos métricas a elección para evaluar la calidad de la partición encontrada. 

Variar el parámetro "resolución" y observar cómo cambia la distribución de comunidades encontradas. ¿Existe algún valor para el cual se identifiquen dos grandes comunidades?

Elegir otro algoritmo de detección de comunidades y comparar los resultados con los obtenidos anteriormente.

**Opcional:** Correr el algoritmo de Louvain con distintas semillas aleatorias. Utilizar alguna métrica de comparación externa entre las particiones obtenidas para determinar en qué medida depende el algoritmo de la condición inicial.

### Segunda parte

### **4. Extracción de etiquetas**

En el archivo [etiquetas.csv](https://drive.google.com/file/d/1LWY3VoIRt0xKwEbbtMXYePOGZvgPsQh-/view?usp=sharing) están las etiquetas para un pequeño subconjunto de nodos. Podemos interpretar el valor de la etiqueta como la pertenencia a una determinada clase, donde los usuarios de una misma clase en general tienden a expresar apoyo entre sí.

- Determinar quiénes son los usuarios referentes de cada clase (utilizar alguna medida de centralidad calculada sobre el grafo de retweets).
- Utiliando los resultados del práctico anterior, determinar si los usuarios de cada clase forman parte de distintas comunidades.

**Opcional:** Reconstruir el archivo "etiquetas.csv". Para eso, hacer lo siguiente

- Construir un grafo en donde los nodos sean usuarios, y donde los enlaces unan dos nodos si entre ellos hubo más respuestas de apoyo que de oposición.
- Extraer las dos componentes más grandes del grafo. Esos serán nuestros nodos etiquetados.

### **5. Embedding de nodos**

- Generar un embedding del grafo de retweets utilizando el algoritmo `word2vec`.
- Reducir a 2 la dimensionalidad del embedding utilizando [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.htmlhttps://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) y [t-SNE](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.htmlhttps://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html).
- Graficar los embeddings correspondientes a los datos etiquetados. ¿Es posible diferenciar unos de otros?

**Opcional:** Graficar además los embeddings de los nodos que forman parte de las comunidades asociadas a cada clase. Determinar si el embedding permite distinguir cada comunidad.

### **Opcional: 6. Redes neuronales de grafos**

El archivo [word_vectors.csv](https://drive.google.com/file/d/1aoxugyMktKb0NQ8Pf3bdhKvh8BAj7YZz/view?usp=sharing) contiene un embedding de 300 dimensiones para cada tweet, otenido utilizando un modelo preentrenado de [FastText](https://fasttext.cc/). Construir una matriz de features para los nodos tomando, para cada usuario, el promedio de los vectores correspondientes a los tweets que escribió. Utilizando estos features, y tomando como ejemplos etiquetados los usuarios de "etiquetas.csv" entrenar una red neuronal de grafos para realizar una clasificación binaria sobre el resto de los nodos. Pueden utilizar como base el siguiente modelo:

In [8]:
pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        torch.manual_seed(1234)
        self.conv1 = GCNConv(dataset.num_features, 4)
        self.conv2 = GCNConv(4, 4)
        self.conv3 = GCNConv(4, 2)
        self.classifier = Linear(2, dataset.num_classes)

    def forward(self, x, edge_index):
        h = self.conv1(x, edge_index)
        h = h.tanh()
        h = self.conv2(h, edge_index)
        h = h.tanh()
        h = self.conv3(h, edge_index)
        h = h.tanh()  # Embedding final
        
        # Aplicamos un clasificador lineal sobre el embedding
        out = self.classifier(h)

        return out, h

NameError: ignored

**Observación:** para alimentar la red neuronal, es necesario construir un objeto de la clase `Dataset` de PyTorch-Geometric. Una forma de hacer eso es la siguiente

In [10]:
from torch_geometric.data import InMemoryDataset, Data

## Reemplazar por el grafo correspondiente
g = nx.Graph()

## Etiquetas. Reemplazar por las clases del archivo 'etiquetas.csv'.
## Asignar la clase '2' a los ejemplos no etiquetados
labels = [1, 0, 2, ..., 1]

## True si el ejemplo está etiquetado (clases 0 y 1)
train_idx = [True, True, False, ..., True]

## Matriz de features (word vectors)
features = ...

adj = nx.to_scipy_sparse_matrix(g).tocoo()
row = torch.from_numpy(adj.row.astype(np.int64)).to(torch.long)
col = torch.from_numpy(adj.col.astype(np.int64)).to(torch.long)
edge_index = torch.stack([row, col], dim=0)


class TwitterDataset(InMemoryDataset):
    def __init__(self, transform=None):
        super(TwitterDataset, self).__init__('.', transform, None, None)

        data = Data(edge_index=edge_index)
        
        data.num_nodes = g.number_of_nodes()
        
        # Features 
        data.x = torch.from_numpy(features).type(torch.float32)
        
        # Etiquetas
        y = torch.from_numpy(labels).type(torch.long)
        data.y = y.clone().detach()
        
        data.num_classes = 2
        
        n_nodes = g.number_of_nodes()
        
        # create train and test masks for data
        train_mask = torch.zeros(n_nodes, dtype=torch.bool)
        train_mask[train_idx] = True
        data['train_mask'] = train_mask

        self.data, self.slices = self.collate([data])

    def _download(self):
        return

    def _process(self):
        return

    def __repr__(self):
        return '{}()'.format(self.__class__.__name__)

ModuleNotFoundError: ignored

# Resolución

## **Ejercicio 1**

Construir el **grafo de retweets**, definido de la siguiente manera:

- Tipo de grafo: Dirigido
- Nodos: ID de los usuarios
- Enlaces: (Usuario A) ---> (Usuario B) si B retweeteó algún tweet de A

Con estos datos, el grafo debería tener alrededor de 40000 nodos y 90000 enlaces.

Considerar la versión no dirigida del grafo y estudiar su conectividad. Si existe una única "componente gigante", realizar el resto de las actividades sobre ella, en lugar de sobre el grafo completo.

Calcular las siguientes métricas globales del grafo:

- Grado medio
- Asortatividad
- Transitividad
- Coeficiente de clustering de Watts-Strogatz

**Opcional:** Comparar las métricas calculadas anteriormente con las de un grafo aleatorio con la misma distribución de grado. Pueden utilizar para ello este [método](https://networkx.org/documentation/stable/reference/generated/networkx.generators.degree_seq.configuration_model.html?highlight=configuration#networkx.generators.degree_seq.configuration_model). Con esto en mente, comentar si los valores obtenidos anteriormente difieren significativamente del caso aleatorio.

### **Construcción del grafo**
Construir el grafo de retweets, definido de la siguiente manera:

Tipo de grafo: Dirigido  
Nodos: ID de los usuarios  
Enlaces: (Usuario A) ---> (Usuario B) si B retweeteó algún tweet de A


**Opcional:** Comparar las métricas calculadas anteriormente con las de un grafo aleatorio con la misma distribución de grado. Pueden utilizar para ello este [método](https://networkx.org/documentation/stable/reference/generated/networkx.generators.degree_seq.configuration_model.html?highlight=configuration#networkx.generators.degree_seq.configuration_model). Con esto en mente, comentar si los valores obtenidos anteriormente difieren significativamente del caso aleatorio.



In [ ]:
import networkx as nx
import numpy as np
G = nx.Graph()
DG = nx.DiGraph()

In [ ]:
df_filter = df[['user.id', 'user_retweeters']]
df_graph = df_filter.explode('user_retweeters').reset_index(drop=True).dropna() # Con el dropeo de los no retweeteados da los valores sugeridos

In [ ]:
G = nx.from_pandas_edgelist(df_graph, source='user.id', target='user_retweeters')
DG = nx.from_pandas_edgelist(df_graph, source='user.id', target='user_retweeters')

Es indistinto para el calculo de nodos y vertices seleccionar el grafo dirigido o el no dirigido

In [ ]:
print(f'Cantidad de nodos: {DG.number_of_nodes()}')
print(f'Cantidad de links: {DG.number_of_edges()}')

Tomamos el subgrafo componente mas grande del grafo no dirigido.

In [ ]:
graph_components = [len(c) for c in sorted(nx.connected_components(G), key=len, reverse=True)]
S = [G.subgraph(c).copy() for c in nx.connected_components(G)]
G_s = S[0] #componente conexa más grande del Grafo no dirigido

In [ ]:
print(f'Cantidad de nodos: {G_s.number_of_nodes()}')
print(f'Cantidad de links: {G_s.number_of_edges()}')

In [ ]:
df_nodes = pd.DataFrame(index=list(G_s.nodes()))
deg_seq = np.array([k for v, k in G_s.degree()])
clustering_coefficient = nx.clustering(G_s)

df_nodes['degree'] = deg_seq
df_nodes['Cws'] = list(clustering_coefficient.values())

In [ ]:
df_nodes = df_nodes.sort_values(by=["degree"], ascending=False)\
            .reset_index().rename(columns={"index" : "user.id"})

### Calculo metricas Globales

Calculamos metricas sobre el subgrafo obtenido anteriormente.

**Grado Medio**

In [ ]:
print(f"El grado medio del Subgrafo global considerado es: {round(df_nodes.degree.mean(),2)} ~ {round(df_nodes.degree.mean())}")

**Asortatividad**

In [ ]:
dac = nx.degree_assortativity_coefficient(G_s)
print(f"Las medidas de similidaridad de las conexiones entre nodos de igual grado es: {round(dac,3)}")

**Transitividad**

In [ ]:
transitivity = nx.transitivity(G_s)
print(f"La transitividad el grafo es: {round(transitivity,5)}")

**Coeficiente medio de Clustering  de Watts-Strogatz**

In [ ]:
print(f"El coeficiente de clustering de Watts-Strogatz es : {round(df_nodes.Cws.mean(),3)}")

### Conclusiones

* Del grafo dirigido obtuvimos que la componente de mayor tamaño tiene el 97% del tamaño del grafo original.
* El grado medio de los nodos de la componente mayor del grafo dirigido da un valor de 5, lo cual indica que en promedio cada usuario retweetea a, o es retweeteado por, otros 5 usuarios.
* Observamos que la asortatividad es negativa y cercana a cero , -0.2, lo que quizá podemos interpretar como que les usuaries que están conectados no necesariamente tienen similaridad entre los temas que retweetean.
* Tanto la transitividad como el coeficiente medio de clustering de WS dieron muy baja, 0.001 y 0.1 respectivamente, por lo que interpretamos que en promedio, hay pocos vecines en común entre vecines.


## **Ejercicio 2**

Calcular 5 métricas de centralidad de nodos. Graficar la distribución de cada una de ellas ¿Existe alguna correlación entre las distintas centralidades? 

Hacer un ranking con los 10 nodos más centrales para cada métrica. ¿Hay coincidencia entre los rankings?. ¿Qué características tienen los usuarios más centrales y sus respectivos tweets?

**Opcional:** Determinar si existe alguna correlación entre la centralidad de un nodo y su actividad en red social. Es decir, evaluar si los usuarios que más escriben son los más centrales o no.

### **Centralidad**

Opcional: Determinar si existe alguna correlación entre la centralidad de un nodo y su actividad en red social. Es decir, evaluar si los usuarios que más escriben son los más centrales o no.

In [ ]:
!pip install igraph

In [ ]:
import igraph as ig

In [ ]:
G_m = ig.Graph.from_networkx(G_s)

In [ ]:
print(f'Cantidad de nodos: {G_m.vcount()}')
print(f'Cantidad de links: {G_m.ecount()}')

In [ ]:
betweenness = G_m.betweenness()
eigenvector = G_m.eigenvector_centrality()
closeness = G_m.closeness()
pagerank = G_m.pagerank()
katz = nx.katz_centrality_numpy(G_s)

In [ ]:
df_nodes['betweenness'] = betweenness
df_nodes['eigenvector'] = eigenvector
df_nodes['closeness'] = closeness
df_nodes['pagerank'] = pagerank
df_nodes['katz'] = katz

df_nodes.head(10)

### Correlacion Medidas de Centralidad

In [ ]:
fig, ax = plt.subplots(figsize=(16,8))
sns.heatmap(df_nodes.corr(), annot=True, cmap= 'Blues', ax=ax)

### Betweenness

Distribucion de la medida de Betwenneess en los nodos.

In [ ]:
fig, ax= plt.subplots(figsize=(20,10))
ax.set_xlabel('betweenness')
ax.hist(df_nodes['betweenness'],bins=25,log=True);

**Top 10 medida de Betweenness**

In [ ]:
df_be = df_nodes.sort_values(by=["betweenness"], ascending=False).head(10)
df_be

### Eigenvector

Distiribucion centralidad de vector propio, en los nodos.

In [ ]:
fig, ax= plt.subplots(figsize=(20,10))
ax.set_xlabel('eigenvector centrality')
ax.hist(df_nodes['eigenvector'],bins=25,log=True);

**Top 10 medida de Centralidad de Vector Propio**

In [ ]:
df_ei = df_nodes.sort_values(by=["eigenvector"], ascending=False).head(10)
df_ei

### Closeness

Distribucion de medida de cercania en los nodos

In [ ]:
fig, ax= plt.subplots(figsize=(20,10))
ax.set_xlabel('closeness')
ax.hist(df_nodes['closeness'],bins=50); #,log=True);

**Top 10 medida de cercania**

In [ ]:
df_cl = df_nodes.sort_values(by=["closeness"], ascending=False).head(10)
df_cl 

### Page Rank

Distribucion de medida de page rank en los nodos.

In [ ]:
fig, ax= plt.subplots(figsize=(20,10))
ax.set_xlabel('pagerank')
ax.hist(df_nodes['pagerank'],bins=20,log=True);

**Top 10 medida de Page Rank**

In [ ]:
df_pr = df_nodes.sort_values(by=["pagerank"], ascending=False).head(10)
df_pr

### Caracteristicas usuarios mas centrales(Terminar)


In [ ]:
#unimos los rankings de las medidas de centralidad
df_tops = pd.concat([df_be,df_cl,df_ei,df_pr])
print(df_tops.shape)
df_tops

In [ ]:
df_tops = df_tops.drop_duplicates(subset=['user.id'])
print(df_tops.shape)
df_tops

In [ ]:
df_tops.describe()

**Usuarios**
 * 152325528: parece un periodista o medio oficialista a favor de la vacunacion.
 * 60982903: vacio
 * 992420860907663361: sentimiento negativo, parece un bot.
 * 2828897977: 

In [ ]:
df[df["user.id"] == "2828897977"].full_text 


### Conclusiones(Terminar)
* Las medidas de centralidad que estan más correlacionadas son: betweeness con page rank (0.97) y con eigenvector (0.73) y eigenvector con page rank (0.78).
* De la unión de los tops 10 de cada medida de centralidad obtuvimos un conjunto de 18 usuaries únicos, es decir que muches de elles se repiten en cada ranking.
* De este conjunto de 18 usuaries vemos que la medida de betweeness y eigenvector tienen mucha dispersión, a diferencia de la closeness, cuyo valores rondan entre 0.33 y 0.39. De la medida de page rank no se encuentra ningún superior a 0.04, por lo que todas son muy cercanas a cero. Finalmente, al ver los grados de cada usuarie, encontramos valores entre 567 y 2, alejado a lo que hubiésemos esperado.

* (viendo los tweets no vemos mucha relacion entre la centralidad y la cantidad de tweets...)


## Ejercicio 3

Utilizar el algoritmo de Louvain con el parámetro "resolución" igual a 1. Caracterizar las comunidades halladas (cantidad, distribución de tamaños). Utilizar la modularidad y otras dos métricas a elección para evaluar la calidad de la partición encontrada.

Variar el parámetro "resolución" y observar cómo cambia la distribución de comunidades encontradas. ¿Existe algún valor para el cual se identifiquen dos grandes comunidades?

Elegir otro algoritmo de detección de comunidades y comparar los resultados con los obtenidos anteriormente.

Opcional: Correr el algoritmo de Louvain con distintas semillas aleatorias. Utilizar alguna métrica de comparación externa entre las particiones obtenidas para determinar en qué medida depende el algoritmo de la condición inicial.
## **Comunidades**

**Opcional:** Correr el algoritmo de Louvain con distintas semillas aleatorias. Utilizar alguna métrica de comparación externa entre las particiones obtenidas para determinar en qué medida depende el algoritmo de la condición inicial.

In [ ]:
!pip install cdlib
from cdlib import NodeClustering, evaluation, algorithms

### Louvian

Aplicamos el algoritmo de Louvian al grafo a los fines de poder caracterizar distintas comunidades, empleando una resolucion igual a 1.

In [ ]:
comms = algorithms.louvain(G_s, weight='weight', resolution=1, randomize=False)

In [ ]:
print(f"{comms.method_name}")
print(f"Parametros del algoritmo : {comms.method_parameters}")
print(f"Numero de comunidades detectadas : {len(comms.communities)}")

Caracterizamos el tamaño de las distintas comunidades.

In [ ]:
communities = {}
for com in range(len(comms.communities)):
    communities[com] = len(comms.communities[com])
df_comms = pd.DataFrame(communities.items(), columns=['Community', 'Size']).sort_values(by=['Size'], ascending=False)
df_comms

### Metricas de Evaluacion Interna algoritmo de Louvian

**Modularidad**

In [ ]:
evaluation.newman_girvan_modularity(G_s, comms)

**Indice de corte**

In [ ]:
evaluation.cut_ratio(G_s, comms)

**Densidad Interna**

In [ ]:
evaluation.internal_edge_density(G_s, comms)

### Comparacion distintas resoluciones

In [ ]:
data = []
resolutions = [0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100]
for resolution in resolutions:
    comms = algorithms.louvain(G_s, weight='weight', resolution=resolution, randomize=False)
    ncomms = len(comms.communities)
    Q = evaluation.newman_girvan_modularity(G_s, comms).score
    mu = evaluation.internal_edge_density(G_s, comms).score
    cut_ratio = evaluation.cut_ratio(G_s, comms).score
    data.append((resolution, ncomms, Q, mu, cut_ratio))

df_Louvian = pd.DataFrame(data, columns=['Resolucion', 'Numero de Comunidades', 'Modularidad', 'Densida Interna', 'Indice de Corte'])

In [ ]:
df_Louvian

No se identifica un valor de resolucion con el cual sea posible obtener dos grandes comunidades con el algoritmo en cuestion.

### Label-propagation

In [ ]:
comms_LP = algorithms.label_propagation(G_s)
Q_LP = evaluation.newman_girvan_modularity(G_s, comms_LP).score
mu_LP = evaluation.internal_edge_density(G_s, comms_LP).score
cut_ratio_LP = evaluation.cut_ratio(G_s, comms_LP).score

In [ ]:
print(f"{comms_LP.method_name}")
print(f"Parametros del algoritmo : {comms_LP.method_parameters}")
print(f"Numero de comunidades detectadas : {len(comms_LP.communities)}")
print(f"Modularidad : {Q_LP}")
print(f"Densidad Interna : {mu_LP}")
print(f"Indice de corte : {cut_ratio_LP}")

### Leiden

In [ ]:
!pip install leidenalg

In [ ]:
comms_Leiden = algorithms.leiden(G_s)
Q_Leiden = evaluation.newman_girvan_modularity(G_s, comms_Leiden).score
mu_Leiden = evaluation.internal_edge_density(G_s, comms_Leiden).score
cut_ratio_Leiden = evaluation.cut_ratio(G_s, comms_Leiden).score

In [ ]:
print(f"{comms_Leiden.method_name}")
print(f"Parametros del algoritmo : {comms_Leiden.method_parameters}")
print(f"Numero de comunidades detectadas : {len(comms_Leiden.communities)}")
print(f"Modularidad : {Q_Leiden}")
print(f"Densidad interna : {mu_Leiden}")
print(f"Indice de corte : {cut_ratio_Leiden}")

### Comparacion Metricas de Evaluacion Interna distintos algoritmos (REVISAR)

In [ ]:
#Grafico para comparar metricas de evaluacion interna
fig, ((ax1, ax2, ax3)) = plt.subplots(3,1, figsize=(20,20))
ax1.set_title("Metricas de evaluacion interna", fontsize=24)
ax1.set_ylabel('Q')
ax1.set_xlabel('Resolution')
ax1.set_xscale('log')
ax1.plot(df_Louvian.Resolucion, df_Louvian.Modularidad, label = 'Louvian')
ax1.axhline(Q_LP, linestyle='--', label='Label Propagation', color='C1')
ax1.axhline(Q_Leiden, linestyle='--', label='Leiden', color='C2')
ax1.legend()

ax2.set_ylabel('$\mu$')
ax2.set_xlabel('Resolution')
ax2.set_xscale('log')
ax2.plot(df_Louvian.Resolucion, df_Louvian["Densida Interna"], label='Louvian')
ax2.axhline(mu_LP, linestyle='--', label='Label Propagation', color='C1')
ax2.axhline(mu_Leiden, linestyle='--', label='Leiden', color='C2')
ax2.legend()

ax3.set_ylabel('Indice de Corte')
ax3.set_xlabel('Resolution')
ax3.set_xscale('log')
ax3.plot(df_Louvian.Resolucion, df_Louvian["Indice de Corte"], label='Louvian')
ax3.axhline(cut_ratio_LP, linestyle='--', label='Label Propagation', color='C1')
ax3.axhline(cut_ratio_Leiden, linestyle='--', label='Leiden', color='C2')
ax3.legend()

Habria que elegir una resolucion del Algoritmo de Louvian para poder comparar el numero de comunidades. 

### Conclusiones(FALTA)

## **Ejercicio 4: Extracción de etiquetas(FALTA)**

En el archivo [etiquetas.csv](https://drive.google.com/file/d/1LWY3VoIRt0xKwEbbtMXYePOGZvgPsQh-/view?usp=sharing) están las etiquetas para un pequeño subconjunto de nodos. Podemos interpretar el valor de la etiqueta como la pertenencia a una determinada clase, donde los usuarios de una misma clase en general tienden a expresar apoyo entre sí.

- Determinar quiénes son los usuarios referentes de cada clase (utilizar alguna medida de centralidad calculada sobre el grafo de retweets).
- Utiliando los resultados del práctico anterior, determinar si los usuarios de cada clase forman parte de distintas comunidades.

**Opcional:** Reconstruir el archivo "etiquetas.csv". Para eso, hacer lo siguiente

- Construir un grafo en donde los nodos sean usuarios, y donde los enlaces unan dos nodos si entre ellos hubo más respuestas de apoyo que de oposición.
- Extraer las dos componentes más grandes del grafo. Esos serán nuestros nodos etiquetados.



..................

## **Ejercicio 5: Embedding de nodos (FALTA)**

- Generar un embedding del grafo de retweets utilizando el algoritmo `word2vec`.
- Reducir a 2 la dimensionalidad del embedding utilizando [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.htmlhttps://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) y [t-SNE](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.htmlhttps://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html).
- Graficar los embeddings correspondientes a los datos etiquetados. ¿Es posible diferenciar unos de otros?

**Opcional:** Graficar además los embeddings de los nodos que forman parte de las comunidades asociadas a cada clase. Determinar si el embedding permite distinguir cada comunidad.

### (Opcional: 6. Redes neuronales de grafos)

El archivo [word_vectors.csv](https://drive.google.com/file/d/1aoxugyMktKb0NQ8Pf3bdhKvh8BAj7YZz/view?usp=sharing) contiene un embedding de 300 dimensiones para cada tweet, otenido utilizando un modelo preentrenado de [FastText](https://fasttext.cc/). Construir una matriz de features para los nodos tomando, para cada usuario, el promedio de los vectores correspondientes a los tweets que escribió. Utilizando estos features, y tomando como ejemplos etiquetados los usuarios de "etiquetas.csv" entrenar una red neuronal de grafos para realizar una clasificación binaria sobre el resto de los nodos. Pueden utilizar como base el siguiente modelo:

..................